In [44]:
import pandas as pd
import numpy as np

In [45]:
class CS():
    def __init__(self, users_req, users_skill, jobs_req, jobs_skill, weight):
        self.users_req = users_req
        self.jobs_req = jobs_req
        self.users_skill = users_skill
        self.jobs_skill = jobs_skill
        self.weight = weight
        self.one_weight = 1-weight
        
        
    def similarity(self):
        row_u_r,col_u_r = self.users_req.shape
        row_u_s,col_u_s = self.users_skill.shape
        row_j_r,col_j_r = self.jobs_req.shape
        row_j_s,col_j_s = self.jobs_skill.shape
        
        from sklearn.metrics.pairwise import cosine_similarity
        similarity_req = cosine_similarity(self.users_req.reshape(row_u_r,col_u_r), self.jobs_req.reshape(row_j_r,col_j_r), dense_output=True)
        
        similarity_skill = cosine_similarity(self.users_skill.reshape(row_u_r,col_u_r), self.jobs_skill.reshape(row_j_r,col_j_r), dense_output=True)
        similarity = similarity_req * self.weight + similarity_skill * self.one_weight
        return similarity
    

In [46]:
class MF():
    
    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.
        
        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """
        
        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))
        
        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])
        
        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]
        
        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 10 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))
        
        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)
            
            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])
            
            # Create copy of row of P since we need to update it but use older values for update on Q
            P_i = self.P[i, :][:]
            
            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * P_i - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction
    
    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return mf.b + mf.b_u[:,np.newaxis] + mf.b_i[np.newaxis:,] + mf.P.dot(mf.Q.T)


In [88]:
weight = 1.0

#df = pd.read_csv('user_data_2.csv')
df1 = pd.read_csv('user_data_2.csv')
df2 = pd.read_csv('user_data_3.csv')
df2 = df2.sort_index(axis = 1)
df = pd.concat([df1, df2], axis=1)
user_data = df.to_numpy()

#df = pd.read_csv('job_data_2.csv')
df1 = pd.read_csv('job_data_2.csv')
df2 = pd.read_csv('job_data_3.csv')
df2 = df2.sort_index(axis = 1)
df = pd.concat([df1, df2], axis=1)
job_data = df.to_numpy()
job_data = np.delete(job_data,0,1)



In [96]:
cs = CS(user_data, user_data,job_data,job_data,weight) 
similarity = cs.similarity()

[[0.08574929 0.1833397  0.         ... 0.         0.         0.10846523]
 [0.         0.31448545 0.2773501  ... 0.2773501  0.13867505 0.12403473]
 [0.125      0.26726124 0.53033009 ... 0.35355339 0.35355339 0.15811388]
 ...
 [0.14744196 0.15762208 0.10425721 ... 0.10425721 0.         0.09325048]
 [0.25       0.40089186 0.35355339 ... 0.35355339 0.1767767  0.        ]
 [0.15430335 0.32991444 0.10910895 ... 0.10910895 0.         0.09759001]]


In [97]:
threshold = 0.32
result = np.transpose((similarity>threshold).nonzero()) 
user_id = -1 
df1 = pd.read_csv('user_data_1.csv') 
df1.fillna(0, inplace = True)
df2 = pd.read_csv('job_data_1.csv') 
df2.fillna(0, inplace = True)
for i in result: 
    if user_id != i[0]: 
        print('-----------------------------------------------')
        print(df1.loc[i[0]]) 
        print('----------------recommendations---------------')
        user_id = i[0] 
    print('\u2022',df2.loc[i[1]])

-----------------------------------------------
ID                                                                    1
location                                                        Germany
Employment            Independent contractor, freelancer, or self-em...
Position              Developer, desktop or enterprise applications;...
EducationLevel          Master’s degree (M.A., M.S., M.Eng., MBA, etc.)
UndergraduateMajor                                                  0.0
Experience                                                           27
Name: 0, dtype: object
----------------recommendations---------------
• ID                                     19
Company                         Genesis10
Employment    Full Time, Direct Placement
Country            Xinyi District, Taiwan
Name: 18, dtype: object
• ID                                          55
Company                       Tellus Solutions
Employment    Contract Corp-To-Corp, 12 Months
Country                      Sydney, Aust

Name: 0, dtype: object
• ID                                  23
Company       Avesta Computer Services
Employment             Full Time, PERM
Country         London, United Kingdom
Name: 22, dtype: object
• ID                          32
Company              Genesis10
Employment           Full Time
Country       Hamburg, Germany
Name: 31, dtype: object
• ID                              54
Company                  Genesis10
Employment               Full Time
Country       Waterloo, ON, Canada
Name: 53, dtype: object
• ID                                     56
Company                         Genesis10
Employment    Full Time, Direct Placement
Country                  München, Germany
Name: 55, dtype: object
• ID                                76
Company        Thinkfind Corporation
Employment                         0
Country       London, United Kingdom
Name: 75, dtype: object
-----------------------------------------------
ID                                                             

Name: 93, dtype: object
----------------recommendations---------------
• ID                                                           12
Company                       CSI (Consultant Specialists Inc.)
Employment    Full Time, Contract Corp-To-Corp, Contract Ind...
Country                              Seattle, WA, United States
Name: 11, dtype: object
• ID                                                16
Company                Eastridge Workforce Solutions
Employment               Full Time, contract-to-hire
Country       South San Francisco, CA, United States
Name: 15, dtype: object
• ID                             21
Company                 Genesis10
Employment              Full Time
Country       Zürich, Switzerland
Name: 20, dtype: object
• ID                                         25
Company       VanderHouwen & Associates, Inc.
Employment               Full Time, Full time
Country          Sunnyvale, CA, United States
Name: 24, dtype: object
• ID                                 

In [98]:
from sklearn.preprocessing import normalize
similarity =normalize(similarity, axis=1, norm='l1')

mf = MF(similarity*5, K=6, alpha=0.1, beta=0.001, iterations=500)
training_process = mf.train()

Iteration: 10 ; error = 2.2277
Iteration: 20 ; error = 2.0465
Iteration: 30 ; error = 1.9544
Iteration: 40 ; error = 1.8279
Iteration: 50 ; error = 1.7193
Iteration: 60 ; error = 1.6099
Iteration: 70 ; error = 1.5301
Iteration: 80 ; error = 1.4667
Iteration: 90 ; error = 1.4199
Iteration: 100 ; error = 1.3882
Iteration: 110 ; error = 1.3542
Iteration: 120 ; error = 1.3265
Iteration: 130 ; error = 1.3175
Iteration: 140 ; error = 1.3096
Iteration: 150 ; error = 1.2792
Iteration: 160 ; error = 1.2769
Iteration: 170 ; error = 1.2634
Iteration: 180 ; error = 1.2521
Iteration: 190 ; error = 1.2477
Iteration: 200 ; error = 1.2417
Iteration: 210 ; error = 1.2350
Iteration: 220 ; error = 1.2474
Iteration: 230 ; error = 1.2264
Iteration: 240 ; error = 1.2302
Iteration: 250 ; error = 1.2346
Iteration: 260 ; error = 1.2278
Iteration: 270 ; error = 1.2271
Iteration: 280 ; error = 1.2164
Iteration: 290 ; error = 1.2194
Iteration: 300 ; error = 1.2246
Iteration: 310 ; error = 1.2260
Iteration: 320 ; 

In [99]:
#print(similarity*5, mf.full_matrix())
prediction = similarity*5-mf.full_matrix()
#print(prediction)
threshold = 0.05
result = np.transpose((prediction < -threshold).nonzero()) 
user_id = -1 
for i in result: 
    if user_id != i[0]: 
        print('-----------------------------------------------')
        print(df1.loc[i[0]]) 
        print('----------------recommendations---------------')
        user_id = i[0] 
    print('\u2022',df2.loc[i[1]])

-----------------------------------------------
ID                                                                    1
location                                                        Germany
Employment            Independent contractor, freelancer, or self-em...
Position              Developer, desktop or enterprise applications;...
EducationLevel          Master’s degree (M.A., M.S., M.Eng., MBA, etc.)
UndergraduateMajor                                                  0.0
Experience                                                           27
Name: 0, dtype: object
----------------recommendations---------------
• ID                                       5
Company                   Matrix Resources
Employment                       Full Time
Country       Sunnyvale, CA, United States
Name: 4, dtype: object
• ID                               6
Company          Yash Technologies
Employment    Full Time, Permanant
Country            Dublin, Ireland
Name: 5, dtype: object
• ID            